In [1]:
# TODO посторить пайплайн работы модели 
#      перестроить весь файл
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from utils.get_sequences import get_sequences
from utils.split_data import split_data
from sklearn.preprocessing import MinMaxScaler
from data_files.data_module import EnergyChickenDataModule
from model_files.model import LSTM
from model_files.trainer import Trainer
from torch.optim import AdamW


RANDOM_SEED = 42
SPLITER_COUNT = 0.8
WINDOW_LENGHT = 100
BATCH_SIZE = 32
MAX_COUNT_DECREASING = 5
LEARNING_RATE = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

df = pd.read_csv('data_files/processed.csv')
df.drop(columns=df.columns[0], inplace=True)
scaler = MinMaxScaler([-1, 1])
data = torch.FloatTensor(scaler.fit_transform(df.iloc[:, 0].to_numpy().reshape((-1, 1)))).to(DEVICE)
sequences = get_sequences(data, WINDOW_LENGHT)
train, test = split_data(sequences, SPLITER_COUNT)

data_module = EnergyChickenDataModule(train, test)
data_module.setup()
model = LSTM(device=DEVICE)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

trainer = Trainer(model=model, 
                  optimizer=optimizer, 
                  loss_function=criterion,
                  n_epochs=WINDOW_LENGHT,
                  device=DEVICE,
                  max_count_decreasing=MAX_COUNT_DECREASING)

train_dataloader = data_module.train_dataloader()
test_dataloader = data_module.test_dataloader()
trainer.upload_data(train_dataloader, test_dataloader)



In [2]:
for i in train_dataloader:
    x = i[0].squeeze()
    print(x.shape)
    print(x)
    break


torch.Size([100])
tensor([-0.3058, -0.2548, -0.1801, -0.0393,  0.5626,  0.5793,  0.5838,  0.4191,
        -0.9838,  0.3672,  0.4412,  0.3489,  0.3084,  0.0252, -0.0429, -0.1757,
        -0.3565, -0.4016, -0.3488, -0.4069, -0.3878, -0.4004, -0.3616, -0.3508,
        -0.3422, -0.3320, -0.2055, -0.1136,  0.3889,  0.4206,  0.2875,  0.3677,
         0.1329,  0.2940,  0.3442,  0.3131,  0.2499, -0.8480,  0.0437,  0.0031,
        -0.2762, -0.3019, -0.3186, -0.2932, -0.3326, -0.3037, -0.3523, -0.3079,
        -0.2724, -0.2440, -0.1751,  0.0741,  0.4982,  0.6292,  0.6590,  0.6375,
         0.4626,  0.5295,  0.5098,  0.4277,  0.2430,  0.0756,  0.0120, -0.2046,
        -0.2768, -0.3070, -0.3162, -0.2965, -0.3088, -0.3216, -0.3147, -0.2762,
        -0.2664, -0.2377, -0.1476, -0.0927,  0.2245,  0.2403,  0.1224,  0.1953,
         0.0368,  0.0625,  0.1251,  0.1514,  0.1427,  0.0093, -0.0488, -0.1178,
        -0.2980, -0.3308, -0.3171, -0.3186, -0.3246, -0.3320, -0.3156, -0.3347,
        -0.2944, -0.27

/home/argen7um/Desktop/code/energy_prediction/data_files/dataset.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (torch.Tensor(sequence), torch.tensor(label).float())


In [3]:
trainer.fit()


tensor([-0.3058, -0.2548, -0.1801, -0.0393,  0.5626,  0.5793,  0.5838,  0.4191,
        -0.9838,  0.3672,  0.4412,  0.3489,  0.3084,  0.0252, -0.0429, -0.1757,
        -0.3565, -0.4016, -0.3488, -0.4069, -0.3878, -0.4004, -0.3616, -0.3508,
        -0.3422, -0.3320, -0.2055, -0.1136,  0.3889,  0.4206,  0.2875,  0.3677,
         0.1329,  0.2940,  0.3442,  0.3131,  0.2499, -0.8480,  0.0437,  0.0031,
        -0.2762, -0.3019, -0.3186, -0.2932, -0.3326, -0.3037, -0.3523, -0.3079,
        -0.2724, -0.2440, -0.1751,  0.0741,  0.4982,  0.6292,  0.6590,  0.6375,
         0.4626,  0.5295,  0.5098,  0.4277,  0.2430,  0.0756,  0.0120, -0.2046,
        -0.2768, -0.3070, -0.3162, -0.2965, -0.3088, -0.3216, -0.3147, -0.2762,
        -0.2664, -0.2377, -0.1476, -0.0927,  0.2245,  0.2403,  0.1224,  0.1953,
         0.0368,  0.0625,  0.1251,  0.1514,  0.1427,  0.0093, -0.0488, -0.1178,
        -0.2980, -0.3308, -0.3171, -0.3186, -0.3246, -0.3320, -0.3156, -0.3347,
        -0.2944, -0.2771, -0.2300, -0.13

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cuda:0 and parameter tensor at cpu